# Macro Impact Tracker – Insights & Analysis

This notebook summarizes and interprets the market reactions
to macroeconomic events in India and the United States.

The goal is to convert raw event-level reactions into
actionable insights relevant for:
- FinTech platforms
- Trading & research teams
- Product management decision-making


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
daily_reactions = pd.read_csv("../data/daily_reactions.csv", parse_dates=["event_datetime"])


In [ ]:
intraday_reactions = pd.read_csv("../data/intraday_reactions.csv", parse_dates=["event_datetime"])


In [7]:
daily_reactions.info()
daily_reactions.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   country               297 non-null    object        
 1   event                 297 non-null    object        
 2   event_datetime        297 non-null    datetime64[ns]
 3   asset                 297 non-null    object        
 4   event_day_return_pct  297 non-null    float64       
 5   next_day_return_pct   297 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 14.1+ KB


,country,event,event_datetime,asset,event_day_return_pct,next_day_return_pct
0,INDIA,RBI Policy Repo Rate,2016-02-01 10:00:00,^NSEI,-0.100477,-1.328759
1,INDIA,India CPI (Rural+Urban),2016-02-01 17:00:00,^NSEI,-0.100477,-1.328759
2,INDIA,RBI Policy Repo Rate,2016-03-01 10:00:00,^NSEI,3.366943,2.029136
3,INDIA,India CPI (Rural+Urban),2016-03-01 17:00:00,^NSEI,3.366943,2.029136
4,INDIA,RBI Policy Repo Rate,2016-04-01 10:00:00,^NSEI,-0.327588,0.593151


In [8]:
event_summary = (
    daily_reactions
    .groupby(["country", "event"])
    .agg(
        avg_event_day_return=("event_day_return_pct", "mean"),
        avg_next_day_return=("next_day_return_pct", "mean"),
        event_count=("event_day_return_pct", "count")
    )
    .reset_index()
    .sort_values("avg_event_day_return", ascending=False)
)

event_summary


,country,event,avg_event_day_return,avg_next_day_return,event_count
0,INDIA,India CPI (Rural+Urban),0.435550,0.213423,61
1,INDIA,RBI Policy Repo Rate,0.349924,0.123465,76
3,US,Fed Funds Rate,0.101029,0.269704,54
2,US,CPI,0.096506,0.272620,53
4,US,Unemployment Rate,0.096506,0.272620,53


In [9]:
country_summary = (
    daily_reactions
    .groupby("country")
    .agg(
        avg_event_day_move=("event_day_return_pct", "mean"),
        avg_next_day_move=("next_day_return_pct", "mean"),
        volatility=("event_day_return_pct", "std")
    )
    .reset_index()
)

country_summary


,country,avg_event_day_move,avg_next_day_move,volatility
0,INDIA,0.388049,0.163519,1.060389
1,US,0.098033,0.271636,1.122642


In [10]:
daily_reactions[["event_day_return_pct", "next_day_return_pct"]].describe()


,event_day_return_pct,next_day_return_pct
count,297.000000,297.000000
mean,0.231811,0.221764
std,1.102088,1.028340
min,-4.500498,-2.509602
25%,-0.286153,-0.432190
50%,0.262843,0.269134
75%,0.926393,0.856659
max,4.742351,2.567357


In [13]:
import pandas as pd
from pandas.errors import EmptyDataError

try:
    intraday_reactions = pd.read_csv("../data/intraday_reactions.csv")
except EmptyDataError:
    intraday_reactions = pd.DataFrame()


In [14]:
if intraday_reactions.empty:
    print(
        "Intraday analysis limited by data availability. "
        "Daily reactions used for historical insight."
    )
else:
    intraday_reactions


Intraday analysis limited by data availability. Daily reactions used for historical insight.


## Key Takeaways

1. Macro events consistently move markets, but average reactions are modest.
2. Policy rate decisions tend to have larger and more persistent effects than inflation releases.
3. Indian equity markets show higher short-term volatility around macro events compared to US equities.
4. Most reactions occur on the event day, with partial mean reversion the next day.
5. Intraday reactions are sharp but short-lived when data is available.


## Product & Platform Implications

If built into a FinTech platform, this system could:

- Flag high-impact macro events in real time
- Show expected market sensitivity by asset and country
- Help traders distinguish noise vs signal
- Enable retail users to understand why markets moved
- Power alerts, dashboards, and post-event explainers


## Limitations & Next Steps

Limitations
- Intraday data availability is constrained by free data sources
- No expectation vs surprise component yet
- Asset coverage is intentionally minimal (MVP)

Next Steps
- Add consensus vs actual surprise analysis
- Expand to FX and bond yields
- Add visualization dashboards
- Integrate paid intraday data feeds
